# LP15
#### Lucía Cañal del Río, Klaudia Jaworek

## 1. Problem Setup

 Three types of excavators can do 4 types of earthworks. The table below provides efficiencies of excavators 
 (in m³ per day of work), numbers of excavators available for work and demands for different types of earthwork (in m³):
 $$
\begin{array}{|c|c|c|c|c|c|}
\hline
\text{Type of excavator} & \text{Efficiency for type I earthwork} & \text{Efficiency for type II earthwork} & \text{Efficiency for type III earthwork} & \text{Efficiency for type IV earthwork} & \text{Number available} \\
\hline
A & 25 & 15 & 16 & 20 & 10 \\
\hline
B & 30 & 10 & 25 & 23 & 16 \\
\hline
C & 24 & 18 & 35 & 31.5 & 15 \\
\hline
\text{Demand} & 320 & 110 & 146 & 220 & - \\
\hline
\end{array}
$$

 Assign numbers of excavators of each type to works in such a way that the exploitation costs are minimized 
 while all the work is done. Daily exploitation costs for the excavators are $290 for excavator A, 
 $240 for excavator B, and $520 for excavator C. Do it using an integer programming model.


## 2. Objective: Minimize the explotation costs

We will use **Branch & Bound** to minimize the explotation costs and determine the quantity of each type of excavators used. Each subproblem will be solved using **Simplex**.

## 3. Branch & Bound for optimization

The method consists of combining systematic exploration of the solution space with techniques to significantly reduce the number of solutions that need to be explored.
The problem is represented as a decision tree. Each node of the tree corresponds to a partial sub-solution or a division of the problem into smaller sub-problems.

## 4. Steps to Implement

### Step 1: Define the objective function

$$
\text{Minimize: } 290 \sum_{j} x_{Aj} + 240 \sum_{j} x_{Bj} + 520 \sum_{j} x_{Cj}
$$

### Step 2: Define the restrictions

To meet the demand for work:
$$
25 x_{AI} + 30 x_{BI} + 24 x_{CI} \geq 320 \quad \text{(Type I job demand)}.
$$
$$
15 x_{AII} + 10 x_{BII} + 18 x_{CII} \geq 110 \quad \text{(Type II job demand)}.
$$
$$
16 x_{AIII} + 25 x_{BIII} + 35 x_{CIII} \geq 146 \quad \text{(Type III job demand)}.
$$
$$
20 x_{AIV} + 23 x_{BIV} + 31.5 x_{CIV} \geq 220 \quad \text{(Type IV job demand)}.
$$

Limit on the number of excavators available:
$$
\sum_{j} x_{Aj} \leq 10 \quad \text{(Type A excavators availables)}.
$$
$$
\sum_{j} x_{Bj} \leq 16 \quad \text{(Type B excavators available)}.
$$
$$
\sum_{j} x_{Cj} \leq 15 \quad \text{(Type C excavators available)}.
$$

Positive and whole solutions: 
$$
x_{ij} \in \mathbb{Z}^{+}.
$$

### Step 3: Iterative Optimization

1. **Solve the Relaxed Problem** 
The initial problem is solved using the simplex solver, which finds an optimal solution. However, the solution may contain fractional values for some decision variables. This means that some excavators are assigned in fractional numbers, which is not allowed in the real-world scenario.
2. **Check for Integer Feasibility** 
If all variables are integers, the solution is feasible, and the algorithm terminates with the current solution.
If some variables are fractional, the solution is not feasible, and the algorithm branches.
3. **Branching** 
Branching involves splitting the problem into two subproblems by fixing one fractional variable to: its lower bound (floor value) and its upper bound (ceil value). These new constraints create two subproblems that are added to a stack for further exploration.
4. **Bounding** 
For each subproblem: solve the subproblem using the simplex solver, calculate the objective value (total cost) and compare the it with the current best cost. If the cost is worse than the current best, the subproblem is discarded. If the cost is better, the solution is kept as the current best solution. If the subproblem is infeasible (violates constraints), it is also pruned.
5. **Iterate Until All Subproblems Are Explored** 
6. **Return the Optimal Solution** 
When all subproblems have been explored or pruned, the algorithm returns: the minimum total cost and the assignment of excavators to tasks that achieves this cost.

## 5. Conclusion

This procedure is a powerful approach as it provides optimal solutions, trying to reduce the computational effort needed by focusing only on the most promising solutions. It avoids the need to explore all possible solutions, although it can still be computationally expensive on problems with large solution spaces.


In [5]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus

# Parameters
costs = {"A": 290, "B": 240, "C": 520} # Daily exploitation costs for each type of excavator
efficiencies = {
    "A": [25, 15, 16, 20], # Efficiency in m3/day for each type of work
    "B": [30, 10, 25, 23],
    "C": [24, 18, 35, 31.5]
}
demand = [320, 110, 146, 220] # Demand (m3) for each type of work
excavator_availability = {"A": 10, "B": 16, "C": 15} # Number of available excavators per type

# Create the initial problem
problem = LpProblem("Minimize_Exploitation_Costs", LpMinimize)

# Decision variables: x[i, j] is the number of excavators of type i assigned to work type j
variables = {
    (excavator, work_type): LpVariable(
        f"x_{excavator}_{work_type+1}", 
        0, 
        excavator_availability[excavator], 
        cat="Continuous" # sets the solving method to simplex
    )
    for excavator in costs.keys()
    for work_type in range(4)
}

# Objective function: minimize total exploitation costs
problem += lpSum(
    costs[excavator] * variables[excavator, work_type]
    for excavator in costs.keys()
    for work_type in range(4)
), "Total_Cost"

# Demand constraints: meet the demand for each type of work
for work_type in range(4):
    problem += lpSum(
        efficiencies[excavator][work_type] * variables[excavator, work_type]
        for excavator in costs.keys()
    ) >= demand[work_type], f"Demand_Work_{work_type+1}"

# Availability constraints: do not exceed the number of available excavators
for excavator in costs.keys():
    problem += lpSum(
        variables[excavator, work_type] for work_type in range(4)
    ) <= excavator_availability[excavator], f"Availability_{excavator}"

# Branch and Bound algorithm with debug messages
def branch_and_bound(problem, max_subproblems=1000):
    best_solution = None
    best_cost = float("inf")
    stack = [problem]
    subproblem_count = 0

    while stack:
        # Stop if the number of subproblems exceeds the limit
        if subproblem_count > max_subproblems:
            print("Exceeded maximum number of subproblems. Stopping.")
            break

        # Pop the current problem
        current_problem = stack.pop()
        current_problem.solve()

        # Check feasibility
        if LpStatus[current_problem.status] != "Optimal":
            continue

        # Get objective value
        current_cost = current_problem.objective.value()

        # Check how is the new value
        if current_cost >= best_cost:
            continue

        # Check if the solution is integer
        is_integer = True
        fractional_vars = []
        for var in current_problem.variables():
            if var.varValue is not None and var.varValue % 1 != 0:
                is_integer = False
                fractional_vars.append(var)

        if is_integer:
            # Update the best solution if necessary
            if current_cost < best_cost:
                best_solution = {v.name: v.varValue for v in current_problem.variables()}
                best_cost = current_cost
                print(f"New best solution found: Cost = {best_cost}")
        else:
            # Branching
            var_to_branch = fractional_vars[0]
            lower_bound = int(var_to_branch.varValue) # Floor of the variable value
            upper_bound = lower_bound + 1 # Ceiling of the variable value

            # Subproblem 1: variable <= floor(value)
            subproblem1 = current_problem.copy()
            subproblem1 += var_to_branch <= lower_bound

            # Subproblem 2: variable >= ceil(value)
            subproblem2 = current_problem.copy()
            subproblem2 += var_to_branch >= upper_bound

            # Add the new subproblems to the stack
            stack.append(subproblem1)
            stack.append(subproblem2)

        # Increment the subproblem counter
        subproblem_count += 1
        print(f"Subproblem {subproblem_count} explored.")

    return best_solution, best_cost

# Apply Branch and Bound to solve the problem
solution, total_cost = branch_and_bound(problem)

# Display the results
if solution:
    print("\n\nMinimum Total Cost:", total_cost)
    for var, value in solution.items():
        print(f"{var}: {value}")
else:
    print("No feasible integer solution found.")


Subproblem 1 explored.
Subproblem 2 explored.
Subproblem 3 explored.
Subproblem 4 explored.
Subproblem 5 explored.
Subproblem 6 explored.
New best solution found: Cost = 10610.0
Subproblem 7 explored.
Subproblem 8 explored.
Subproblem 9 explored.
Subproblem 10 explored.
Subproblem 11 explored.
Subproblem 12 explored.
Subproblem 13 explored.
Subproblem 14 explored.
Subproblem 15 explored.
Subproblem 16 explored.
New best solution found: Cost = 10380.0
Subproblem 17 explored.
Subproblem 18 explored.
Subproblem 19 explored.
Subproblem 20 explored.
Subproblem 21 explored.
Subproblem 22 explored.
Subproblem 23 explored.
Subproblem 24 explored.
Subproblem 25 explored.
Subproblem 26 explored.
Subproblem 27 explored.
Subproblem 28 explored.
New best solution found: Cost = 10320.0
Subproblem 29 explored.
Subproblem 30 explored.
Subproblem 31 explored.
Subproblem 32 explored.
Subproblem 33 explored.
Subproblem 34 explored.
Subproblem 35 explored.
Subproblem 36 explored.
Subproblem 37 explored.
S